In [1]:
# Sử dụng tập lệnh mã thông báo chính thức do nhóm Google tạo
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

In [2]:
!pip install tokenization

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub

import tokenization


import re
import string
import nltk
from nltk.corpus import stopwords


from sklearn import model_selection
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV,StratifiedKFold,RandomizedSearchCV


import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")

import warnings
warnings.filterwarnings('ignore')

# Load and Prepare Data

In [4]:
#Training data
train = pd.read_csv('train.csv')
print('Kiểm tra dạng dữ liệu: ', train.shape)
train.head()

Kiểm tra dạng dữ liệu:  (7613, 5)


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
# Testing data
test = pd.read_csv('test.csv')
print('Kiểm tra dạng dữ liệu: ', test.shape)
test.head()

Kiểm tra dạng dữ liệu:  (3263, 4)


,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


# EDA

In [6]:
#Giá trị thiếu trong tập huấn luyện
train.isnull().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [7]:
#Giá trị thiếu trong tập kiểm tra
test.isnull().sum()

id             0
keyword       26
location    1105
text           0
dtype: int64

In [8]:
train['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

#Traditional NLP - Bag of Words + Linear Model

#Tiền xử lý dữ liệu

In [9]:
# lấy bản sao dữ liệu để lại bản gốc cho BERT
train1 = train.copy()
test1 = test.copy()

In [10]:
# Áp dụng vòng kỹ thuật làm sạch văn bản đầu tiên

def clean_text(text):
    '''Viết chữ thường, xóa văn bản trong ngoặc vuông, xóa liên kết, xóa dấu câu và xóa các từ có chứa số.'''
    text = text.lower() # làm cho văn bản viết thường
    text = re.sub('\[.*?\]', '', text) # xóa văn bản trong ngoặc vuông
    text = re.sub('https?://\S+|www\.\S+', '', text) # Xoá URLs
    text = re.sub('<.*?>+', '', text) # Xoá html tags
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text) # xóa dấu câu
    text = re.sub('\n', '', text) # loại bỏ các từ chứa số
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[‘’“”…]', '', text)

    return text

In [11]:
#Xoá bỏ biểu tượng cảm xúc
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

# Áp dụng chức năng de=emoji cho cả tập dữ liệu kiểm tra và tập dữ liệu huấn luyện
train1['text'] = train1['text'].apply(lambda x: remove_emoji(x))
test1['text'] = test1['text'].apply(lambda x: remove_emoji(x))

In [12]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [13]:
#Hàm tiền xử lý văn bản
def text_preprocessing(text):
    """
    Làm sạch và phân tích văn bản.

    """
    tokenizer_reg = nltk.tokenize.RegexpTokenizer(r'\w+')

    nopunc = clean_text(text)
    tokenized_text = tokenizer_reg.tokenize(nopunc)
    remove_stopwords = [w for w in tokenized_text if w not in stopwords.words('english')]
    combined_text = ' '.join(remove_stopwords)
    return combined_text

# Áp dụng chức năng làm sạch cho cả tập dữ liệu kiểm tra và tập dữ liệu huấn luyện
train1['text'] = train1['text'].apply(lambda x: text_preprocessing(x))
test1['text'] = test1['text'].apply(lambda x: text_preprocessing(x))

#Xem văn bản được cập nhật
train1['text'].head()

0         deeds reason earthquake may allah forgive us
1                forest fire near la ronge sask canada
2    residents asked shelter place notified officer...
3    people receive wildfires evacuation orders cal...
4    got sent photo ruby alaska smoke wildfires pou...
Name: text, dtype: object

# Bag of Words Vectorizer

In [14]:
#count_vectorizer = CountVectorizer()
count_vectorizer = CountVectorizer(ngram_range = (1,1), min_df = 1)
train_vectors = count_vectorizer.fit_transform(train1['text'])
test_vectors = count_vectorizer.transform(test1["text"])

# Chỉ giữ lại các phần tử khác 0 để bảo toàn không gian
train_vectors.shape

(7613, 16412)

# TF-IDF Vectorizer

Ở đây, Sử dụng 1 và 2 gam trong đó mỗi thuật ngữ phải xuất hiện ít nhất hai lần và bỏ qua các thuật ngữ xuất hiện trong hơn 50% ví dụ văn bản.

In [15]:
tfidf = TfidfVectorizer(ngram_range=(1, 2), min_df = 2, max_df = 0.5)
train_tfidf = tfidf.fit_transform(train1['text'])
test_tfidf = tfidf.transform(test1["text"])

train_tfidf.shape

(7613, 11077)

In [16]:
# Fitting Logistic Regression đơn giản trên Bow
logreg_bow = LogisticRegression(C=1.0)
scores = model_selection.cross_val_score(logreg_bow, train_vectors, train["target"], cv=5, scoring="f1")
scores.mean()

0.5834476966398702

In [17]:
# Fitting Logistic Regression đơn giản trên TFIDF
logreg_tfidf = LogisticRegression(C=1.0)
scores = model_selection.cross_val_score(logreg_tfidf, train_tfidf, train["target"], cv=5, scoring="f1")
scores.mean()

0.5451346913743611

In [18]:
# Fitting Naive Bayes đơn giản trên BoW
NB_bow = MultinomialNB()
scores = model_selection.cross_val_score(NB_bow, train_vectors, train["target"], cv=5, scoring="f1")
scores.mean()

0.6584930948850116

In [19]:
# Fit Naive Bayes đơn giản trên TFIDF
NB_tfidf = MultinomialNB()
scores = model_selection.cross_val_score(NB_tfidf, train_tfidf, train["target"], cv=5, scoring="f1")
scores.mean()

0.6187711183101462

In [20]:
NB_bow.fit(train_vectors, train["target"])

MultinomialNB()

In [21]:
sample_submission = pd.read_csv('sample_submission.csv')
sample_submission["target"] = NB_bow.predict(test_vectors)

sample_submission.to_csv("submission1.csv", index=False)

# BERT Model

# Helper Functions

In [22]:
# Hàm Mã hóa lấy cột văn bản từ khung dữ liệu huấn luyện hoặc kiểm tra, mã thông báo,
# và độ dài tối đa của chuỗi văn bản làm đầu vào.

#Đầu ra:
#Token
# Mặt nạ đệm - BERT học bằng cách che các mã thông báo nhất định trong mỗi chuỗi.
# id phân đoạn

def bert_encode(texts, tokenizer, max_len = 512):
    all_tokens = []
    all_masks = []
    all_segments = []

    for text in texts:
        text = tokenizer.tokenize(text)

        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)

        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len

        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)

    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [23]:
# Xây dựng và biên dịch mô hình

def build_model(bert_layer, max_len = 512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(clf_output)

    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

    return model

# Load and Prepare Data

In [24]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
submission = pd.read_csv("sample_submission.csv")

In [25]:
 def decontracted(phrase):
     # specific
     phrase = re.sub(r"won\'t", "will not", phrase)
     phrase = re.sub(r"can\'t", "can not", phrase)

     # general
     phrase = re.sub(r"n\'t", " not", phrase)
     phrase = re.sub(r"\'re", " are", phrase)
     phrase = re.sub(r"\'s", " is", phrase)
     phrase = re.sub(r"\'d", " would", phrase)
     phrase = re.sub(r"\'ll", " will", phrase)
     phrase = re.sub(r"\'t", " not", phrase)
     phrase = re.sub(r"\'ve", " have", phrase)
     phrase = re.sub(r"\'m", " am", phrase)
     return phrase

In [26]:
pip install spacy

In [27]:
import spacy

!python -m spacy download en_core_web_sm

# Tải mô hình 'en_core_web_sm'
nlp = spacy.load("en_core_web_sm")

def preprocessing(text):
    text = text.replace('#','')
    text = decontracted(text)
    text = re.sub('\S*@\S*\s?','',text)
    text = re.sub('http[s]?:(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','',text)

    token=[]
    result=''
    text = re.sub('[^A-z]', ' ',text.lower())

    text = nlp(text)
    for t in text:
        if not t.is_stop and len(t)>2:
            token.append(t.lemma_)
    result = ' '.join([i for i in token])

    return result.strip()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 26.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [28]:
 train.text = train.text.apply(lambda x : preprocessing(x))
 test.text = test.text.apply(lambda x : preprocessing(x))

In [29]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,deed reason earthquake allah forgive,1
1,4,NaN,NaN,forest fire near ronge sask canada,1
2,5,NaN,NaN,resident ask ishelter place notify officer eva...,1
3,6,NaN,NaN,people receive wildfire evacuation order calif...,1
4,7,NaN,NaN,get send photo ruby alaska smoke wildfires pou...,1


# Modelling

In [30]:
# Tải xuống kiến ​​trúc BERT
# BERT-Large không phân lớp: 24 lớp, 1024 nút ẩn, 16 đầu chú ý, tham số 340M

module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

In [31]:
!pip install transformers

In [32]:
from transformers import BertTokenizer

# Lấy đường dẫn tới file vocab của BERT và cờ do_lower_case
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()

# Tạo tokenizer từ BertTokenizer
tokenizer = BertTokenizer(vocab_file, do_lower_case)

In [33]:
# Tiền xử lý văn bản
def preprocess_text(text):
    # Xử lý các kí tự đặc biệt, chuyển về lower case
    text = text.lower().strip()
    # Tokenize văn bản sử dụng tokenizer
    tokens = tokenizer.tokenize(text)
    # Chuyển tokens thành IDs
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    return token_ids


In [34]:
train_input = bert_encode(train.text.values, tokenizer, max_len=160)
test_input = bert_encode(test.text.values, tokenizer, max_len=160)

train_labels = train.target.values

In [35]:
model = build_model(bert_layer, max_len=160)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_word_ids (InputLayer  [(None, 160)]                0         []                            
 )                                                                                                
                                                                                                  
 input_mask (InputLayer)     [(None, 160)]                0         []                            
                                                                                                  
 segment_ids (InputLayer)    [(None, 160)]                0         []                            
                                                                                                  
 keras_layer (KerasLayer)    [(None, 1024),               3351418   ['input_word_ids[0][0]',  

In [36]:
checkpoint = ModelCheckpoint('model.h5', monitor='val_loss', save_best_only=True)

train_history = model.fit(
    train_input, train_labels,
    validation_split=0.2,
    epochs=5,
    callbacks=[checkpoint],
    batch_size=16
)

Epoch 1/5
381/381 [==============================] - 794s 2s/step - loss: 0.8449 - accuracy: 0.5374 - val_loss: 0.6916 - val_accuracy: 0.5345
Epoch 2/5
381/381 [==============================] - 651s 2s/step - loss: 0.6971 - accuracy: 0.5494 - val_loss: 0.6955 - val_accuracy: 0.4655
Epoch 3/5
381/381 [==============================] - 651s 2s/step - loss: 0.6940 - accuracy: 0.5491 - val_loss: 0.7067 - val_accuracy: 0.5345
Epoch 4/5
381/381 [==============================] - 714s 2s/step - loss: 0.6881 - accuracy: 0.5606 - val_loss: 0.6913 - val_accuracy: 0.5345
Epoch 5/5
381/381 [==============================] - 651s 2s/step - loss: 0.6874 - accuracy: 0.5567 - val_loss: 0.6961 - val_accuracy: 0.5345


# Predictions and Submission

In [37]:
model.load_weights('model.h5')
test_pred = model.predict(test_input)

102/102 [==============================] - 114s 1s/step


In [38]:
submission['target'] = test_pred.round().astype(int)
submission.head()

submission.to_csv("submission2.csv", index=False)